In [ ]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import shutil
import random
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def build(path):
  newpath = path + "/data"


  if not os.path.exists(newpath):
    print("creating new directory...")

    os.makedirs(newpath)

    path1 = path + "/downSyndrome"
    path2 = path + "/healty"


    files = os.listdir(path1)
    for file in files:
      shutil.copy2(os.path.join(path1, file), newpath)


    files = os.listdir(path2)
    for file in files:
      shutil.copy2(os.path.join(path2, file), newpath)

  else:
    print("data already exists")


  data = os.listdir(newpath)
  random.shuffle(data)
  split_1 = int(0.8 * len(data))
  split_2 = int(0.9 * len(data))
  train = data[:split_1]
  valid = data[split_1:split_2]
  test = data[split_2:]


  return train, valid, test

In [ ]:
path = "/content/drive/MyDrive/archive"
data = build(path)


train_labels = []
val_labels = []
test_labels = []


train, val, test = data

creating new directory...


In [ ]:
for path in train:
    if "down" in path:
      train_labels.append(1)
    else:
      train_labels.append(0)

In [ ]:
for path in test:
  if "down" in path:
    test_labels.append(1)
  else:
    test_labels.append(0)

In [ ]:
for path in val:
  if "down" in path:
    val_labels.append(1)
  else:
    val_labels.append(0)

In [ ]:
from PIL import Image

base_path = '/content/drive/MyDrive/archive/processed_data/'

if not os.path.exists(base_path):
  os.makedirs(base_path)

In [ ]:
def save_processed_imgs(images, path, labels):
  data_path = base_path + path
  if not os.path.exists(data_path):
    os.makedirs(data_path)

  downSyndrome_path = os.path.join(data_path, "downSyndrome")
  healthy_path = os.path.join(data_path, "healthy")

  if not os.path.exists(downSyndrome_path):
    os.makedirs(downSyndrome_path)
  if not os.path.exists(healthy_path):
    os.makedirs(healthy_path)

  for index, image in enumerate(images):
    if image is None or image.size == 0:
      print(f"Skipping image {index} due to zero size.")
      continue

    if index >= len(labels):
      print(f"Skipping image {index} because label does not exist.")
      continue

    normalized_image = (image - image.min()) / (image.max() - image.min()) * 255
    im = Image.fromarray(normalized_image.astype('uint8'))


    if labels[index] == 0:
      im.save(os.path.join(healthy_path, f'{index}.png'))
    else:
      im.save(os.path.join(downSyndrome_path, f'{index}.png'))

In [ ]:
# For SSD
def ssd(dir, image):
  image = cv2.imread(dir + image)
  detector = cv2.dnn.readNetFromCaffe("/content/deploy.prototxt.txt","/content/res10_300x300_ssd_iter_140000.caffemodel")
  img_blob = cv2.dnn.blobFromImage(image)

  detector.setInput(img_blob)
  detections = detector.forward()
  detections = detections[0][0]


  df = pd.DataFrame(detections, columns=["img_id", "is_face", "confidence", "left", "top", "right", "bottom"])
  df = df[df["is_face"]==1]
  df = df[df["confidence"] > 0.9]


  for index, instance in df.iterrows():
    left = int(instance['left'] * 300)
    right = int(instance['right'] * 300)
    top = int(instance['top'] * 300)
    bottom = int(instance['bottom'] * 300)


    if bottom <= 300 and right <= 300:
      image = image[top:bottom, left:right]
    return image

In [ ]:
ssd_train = []
ssd_val = []
ssd_test = []
dir = "/content/drive/MyDrive/archive/data/"

data_dir = '/content/drive/MyDrive/archive/processed_data/'

if not os.path.exists(data_dir):
  for image in train:
    if image is not None:
      ssd_train.append(ssd(dir, image))
  save_processed_imgs(ssd_train, 'train', train_labels)

  for image in val:
    if image is not None:
      ssd_val.append(ssd(dir, image))
  save_processed_imgs(ssd_val, 'val', val_labels)

  for image in test:
    if image is not None:
      ssd_test.append(ssd(dir, image))
  save_processed_imgs(ssd_test, 'test', test_labels)

else:
  print("processed images already present.")

Skipping image 52 due to zero size.
Skipping image 123 due to zero size.
Skipping image 237 due to zero size.
Skipping image 368 due to zero size.
Skipping image 560 due to zero size.
Skipping image 604 due to zero size.
Skipping image 622 due to zero size.
Skipping image 657 due to zero size.
Skipping image 696 due to zero size.
Skipping image 704 due to zero size.
Skipping image 898 due to zero size.
Skipping image 955 due to zero size.
Skipping image 1100 due to zero size.
Skipping image 1109 due to zero size.
Skipping image 1157 due to zero size.
Skipping image 1206 due to zero size.
Skipping image 1375 due to zero size.
Skipping image 1548 due to zero size.
Skipping image 1566 due to zero size.
Skipping image 1667 due to zero size.
Skipping image 1684 due to zero size.
Skipping image 1750 due to zero size.
Skipping image 1916 due to zero size.
Skipping image 1927 due to zero size.
Skipping image 2054 due to zero size.
Skipping image 2093 due to zero size.
Skipping image 2115 due t

In [ ]:
# For MTCNN
import dlib
from imutils import paths
from mtcnn.mtcnn import MTCNN
import face_recognition


def mtcnn(dir, image):
  image = face_recognition.load_image_file(dir + image)

  detector = MTCNN()
  face_locations = detector.detect_faces(image)

  for face in zip(face_locations):
    x,y,w,h = face[0]['box']
    landmarks = face[0]

  image = image[y:y+h, x:x+w]
  return image

RuntimeError: Error while calling cudaGetDevice(&the_device_id) in file /tmp/pip-install-nj6k7_al/dlib_dd113b177fa647df963e4f896c088a8b/dlib/cuda/gpu_data.cpp:204. code: 35, reason: CUDA driver version is insufficient for CUDA runtime version

In [ ]:
mtcnn_train = []
mtcnn_val = []
mtcnn_test = []
dir = "/content/drive/MyDrive/archive/data/"


data_dir = '/content/drive/MyDrive/archive/processed_data/'


if not os.path.exists(data_dir):
  for image in train:
    if image is not None:
      mtcnn_train.append(mtcnn(dir, image))
  save_processed_imgs(mtcnn_train, 'train', train_labels)

  for image in val:
    if image is not None:
      mtcnn_val.append(mtcnn(dir, image))
  save_processed_imgs(mtcnn_val, 'val', val_labels)

  for image in test:
    if image is not None:
      mtcnn_test.append(mtcnn(dir, image))
  save_processed_imgs(mtcnn_test, 'test', test_labels)

else:
  print("processed images already present.")

processed images already present.


In [ ]:
#Classification Model
from keras.preprocessing.image import img_to_array
import tensorflow as tf

In [ ]:
def normalise(image):
  image = cv2.resize(image, (224, 224))

  if len(image.shape) == 2: # If image is grayscale, convert to RGB
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

  image = img_to_array(image)
  image = image / 255.0
  return image

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen=ImageDataGenerator(preprocessing_function = normalise)
validation_datagen = ImageDataGenerator(preprocessing_function = normalise)

train_generator = train_datagen.flow_from_directory(
directory="/content/drive/MyDrive/archive/processed_data/train",
target_size=(224, 224),
batch_size=8,
class_mode="binary"
)

val_generator = validation_datagen.flow_from_directory(
directory="/content/drive/MyDrive/archive/processed_data/val",
target_size=(224, 224),
batch_size=8,
class_mode="binary"
)

early_stopping = tf.keras.callbacks.EarlyStopping(
monitor="val_accuracy",
min_delta=0.05,
patience=15,
verbose=1,
mode="max",
baseline=None,
restore_best_weights=True
)

METRICS = [
tf.keras.metrics.BinaryAccuracy(name='accuracy'),
tf.keras.metrics.Precision(name='precision'),
tf.keras.metrics.Recall(name='recall')
]

Found 2365 images belonging to 2 classes.
Found 293 images belonging to 2 classes.


In [ ]:
# Using ResNet50
import os

path_checkpoint = "model_cp_resnet.ckpt"
directory_checkpoint = os.path.dirname(path_checkpoint)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=path_checkpoint,
save_weights_only=True,
verbose=1)

# Define input layer
input_layer = tf.keras.layers.Input(shape=(224, 224, 3), name='image_input')

# Create ResNet50 Model
resnet = tf.keras.applications.ResNet50(
include_top=False,
weights='imagenet',
input_tensor=input_layer,
pooling=None
)

# Freeze ResNet50 layers
for layer in resnet.layers:
  layer.trainable = False

# Add custom classification layers
x = tf.keras.layers.GlobalAveragePooling2D()(resnet.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(8, activation='relu')(x)
predictions = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# Create final model
model = tf.keras.models.Model(inputs=input_layer, outputs=predictions)

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
from keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.0001), loss='binary_crossentropy', metrics=METRICS)

history = model.fit_generator(
train_generator,
steps_per_epoch=len(train_generator),
epochs=100,
validation_data=val_generator,
validation_steps=len(val_generator),
callbacks=[early_stopping]
)

Epoch 1/100


<ipython-input-20-273ea5570688>:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


296/296 [==============================] - 461s 2s/step - loss: 0.6937 - accuracy: 0.5031 - precision: 0.5034 - recall: 0.9971 - val_loss: 0.6935 - val_accuracy: 0.4608 - val_precision: 0.4608 - val_recall: 1.0000
Epoch 2/100
296/296 [==============================] - 464s 2s/step - loss: 0.6932 - accuracy: 0.5040 - precision: 0.5040 - recall: 0.9992 - val_loss: 0.6935 - val_accuracy: 0.4608 - val_precision: 0.4608 - val_recall: 1.0000
Epoch 3/100
296/296 [==============================] - 464s 2s/step - loss: 0.6932 - accuracy: 0.5044 - precision: 0.5042 - recall: 0.9992 - val_loss: 0.6935 - val_accuracy: 0.4608 - val_precision: 0.4608 - val_recall: 1.0000
Epoch 4/100
296/296 [==============================] - 461s 2s/step - loss: 0.6931 - accuracy: 0.5061 - precision: 0.5051 - recall: 0.9924 - val_loss: 0.6936 - val_accuracy: 0.4608 - val_precision: 0.4608 - val_recall: 1.0000
Epoch 5/100
296/296 [==============================] - 426s 1s/step - loss: 0.6932 - accuracy: 0.5040 - prec

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.legend()

In [ ]:
# Using custom CNN Model

import os

path_checkpoint = "model_cp_custom.ckpt"
directory_checkpoint = os.path.dirname(path_checkpoint)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=path_checkpoint,
save_weights_only=True,
verbose=1)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten()) # this converts our 3D feature maps to 1D feature vectors
model.add (Dense (64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
optimizer='rmsprop' ,
metrics=METRICS)

In [ ]:
history = model.fit_generator(
train_generator,
steps_per_epoch=250,
epochs=100,
validation_data=val_generator,
validation_steps=20,
callbacks=[early_stopping]
)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.legend()